In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def render_scatter_w_tables(metrics_df):
    # Create a figure with 2 rows and 1 column
    fig = make_subplots(rows=3, cols=2, specs=[[{'colspan':2},None],
                                            [{"type": "table",
                                                'rowspan':2},{"type": "table",
                                                'rowspan':2}],
                                            [None,None]],
                                            subplot_titles=['','Top 12 LBs',
                                                            'Top 12 DBs'])

    # Scatter plot
    dict_colors = {0:'blue',1:'red',2:'brown',3:'lightcoral'}
    for i in range(4):
        metrics_quartile = metrics_df[metrics_df['Quartiles'] == i]
        hover_text = metrics_quartile.apply(lambda row: f"{row['displayName_defender']} - Position {row['position']}", axis=1)
        scatter_trace = go.Scatter(x=metrics_quartile['TtTOE'], y=metrics_quartile['TAR+'], mode='markers', text=hover_text, showlegend=True,
                                    marker=dict(color=dict_colors[i],size = 6,opacity = 1), name= f'Quartile {i+1}'
                                    )
        fig.add_trace(scatter_trace, row=1, col=1)

    # Add horizontal and vertical lines to Scatter plot
    # fig.add_hline(y=metrics_df['TAR+'].mean(), line_color='red', line_width=2, line_dash="dash", name='Mean TAR+')
    # fig.add_vline(x=metrics_df['TtTOE'].mean(), line_color='red', line_width=2, line_dash="dash", name='Mean TtTOE')

    # Set explicit range for x and y axes for Scatter plot
    fig.update_xaxes(range=[metrics_df['TtTOE'].min() - 0.5, metrics_df['TtTOE'].max() + 0.5], showgrid=False, row=1, col=1)
    fig.update_yaxes(range=[metrics_df['TAR+'].min() - 0.2, metrics_df['TAR+'].max() + 0.2], showgrid=False, row=1, col=1)

    # Add annotations for above and below the median of 'TAR+' to Scatter plot
    median_tarplus = metrics_df['TAR+'].median()
    fig.add_annotation(
        x=50,
        y=median_tarplus + 1,
        text='Above median tacklers',
        showarrow=False,
        ax=0,
        ay=-40,
        font=dict(size=20, color='black', family='Arial'),
        row=1, col=1
    )

    fig.add_annotation(
        x=50,
        y=median_tarplus - 1,
        text='Below median tacklers',
        showarrow=False,
        ax=0,
        ay=40,
        font=dict(size=20, color='black', family='Arial'),
        row=1, col=1
    )
    # Set explicit range for x and y axes
    fig.update_layout(
        xaxis=dict(range=[metrics_df['TtTOE'].min() - 0.5, metrics_df['TtTOE'].max() + 0.5], showgrid=False),
        yaxis=dict(range=[metrics_df['TAR+'].min() - 0.2, metrics_df['TAR+'].max() + 0.2], showgrid=False),
    )

    # Add annotations for above and below the median of 'TAR+'
    median_tarplus = metrics_df['TAR+'].median()

    fig.add_annotation(
        x=50,
        y=median_tarplus + 1,
        text='Above median tacklers',
        showarrow=False,
        ax=0,
        ay=-40,
        font=dict(size=20, color='black', family='Arial'),
    )

    fig.add_annotation(
        x=50,
        y=median_tarplus - 1,
        text='Below median tacklers',
        showarrow=False,
        ax=0,
        ay=40,
        font=dict(size=20, color='black', family='Arial'),
    )

    selected_players = ['Chauncey Gardner-Johnson', 'Marcus Williams', 'Darious Williams', 'John Johnson', 'Bobby Wagner',
                        'Kristian Fulton','Jordan Hicks']

    annotations = []

    for player in selected_players:
        metrics_player = metrics_df[metrics_df['displayName_defender'] == player]
        dict_ann = dict(
            x=metrics_player['TtTOE'].values[0],
            y=metrics_player['TAR+'].values[0],
            text=f"{player} Position: {metrics_player['position'].values[0]}",
            showarrow=True,
            arrowhead=1,
            arrowsize=1,
            arrowwidth=1,
            arrowcolor='black',
            ax=0,
            ay=-10,
            font=dict(size=12, color='black', family='Arial')
        )
        annotations.append(dict_ann)

    for annotation in annotations:
        fig.add_annotation(**annotation)

    db_df = metrics_df[metrics_df['Position Group'] == 'DB'].sort_values('TAR + TtTOE', ascending=False).head(12)[
        ['displayName_defender', 'position','TtTOE scaled',
        'TAR+ scaled', 'TAR + TtTOE']].round(2)
    lb_df = metrics_df[metrics_df['Position Group'] == 'DB'].sort_values('TAR + TtTOE', ascending=False).head(12)[
        ['displayName_defender', 'position','TtTOE scaled',
        'TAR+ scaled', 'TAR + TtTOE']].round(2)

    # Add tables to subplot
    fig.add_trace(go.Table(header=dict(values=['Player', 'Position','TtTOAR percentile',
                                                'TAR+ percentile', 'Composite Score'],
                            fill_color=dict_colors[0],
                            align='center',
                            font=dict(color='white', size=12)),
                        cells=dict(values=[db_df[col] for col in db_df.columns],
                                    height=24,
                                    font_size=10,
                                    align='left')
                        ), row=2, col=1)

    fig.add_trace(go.Table(header=dict(values=['Player', 'Position','TtTOAR percentile',
                                                'TAR+ percentile', 'Composite Score'],
                            fill_color=dict_colors[0],
                            align='center',
                            font=dict(color='white', size=12)),
                        cells=dict(values=[db_df[col] for col in lb_df.columns],
                                    height=24,
                                    font_size=10,
                                    align='left')
                        ), row=2, col=2)

    # Update layout
    fig.update_layout(
        height=800,
        title_text="Estimated time to tackle opportuniy (eTTO) vs Tackling Above Replacement (TAR+)",
        title_font=dict(size=24, family='Arial', color='black'),
        xaxis_title="eTTO",
        yaxis_title="TAR+",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
    )

    return fig